In [4]:
from tensorflow.keras import layers
import tensorflow as tf

In [19]:
#自定义线性层
class Linear(layers.Layer): 
  def __init__(self, units=32, input_dim=28):
    super(Linear, self).__init__()
    w_init = tf.random_normal_initializer()  #随机的符合正态分布的随机数
    self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),dtype='float32'),trainable=False)
    # 一起写为下
    #self.w = self.add_weight(shape=(input_dim, units),initializer='random_normal',trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(initial_value=b_init(shape=(units,),dtype='float32'),trainable=False)

  def call(self, inputs): 
    return tf.matmul(inputs, self.w) + self.b

In [21]:
inputs = tf.ones((2, 2)) 
linear_layer = Linear(4, 2)
print(linear_layer.w) 
# outputs = linear_layer(inputs)  # 这一步，底层调用了call函数
# print(linear_layer.w)

# 这边初始化后，之后生成w和b，

<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[-0.05198208,  0.01210876, -0.01082835, -0.01559412],
       [-0.04202024,  0.01737851, -0.03601163, -0.00970178]],
      dtype=float32)>


In [23]:
##########   input_dim未知  ################
class Linear(layers.Layer):

  def __init__(self, units=32):
    super(Linear, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units),
                             initializer='random_normal',
                             trainable=True)
    self.b = self.add_weight(shape=(self.units,),
                             initializer='random_normal',
                             trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [24]:
inputs = tf.ones((2, 2))
linear_layer = Linear(4)
outputs = linear_layer(inputs)
print(linear_layer.w)

# 这边没有调用call的话，没有初始化w和b

<tf.Variable 'linear_7/Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[ 0.04887731, -0.00444258, -0.08269116, -0.05387871],
       [ 0.01853305,  0.06927157, -0.04344487,  0.02797734]],
      dtype=float32)>


In [25]:
#模型
class MLPBlock(layers.Layer):

  def __init__(self):
    super(MLPBlock, self).__init__()
    self.linear_1 = Linear(32)
    self.linear_2 = Linear(32)
    self.linear_3 = Linear(1)

  def call(self, inputs):
    x = self.linear_1(inputs)
    x = tf.nn.relu(x)
    x = self.linear_2(x)
    x = tf.nn.relu(x)
    return self.linear_3(x)

In [30]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
# train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
# print("train_images shape", train_images.shape)
train_images = (train_images - 127.5) / 127.5 # 将图片标准化到 [-1, 1] 区间内
print("train_images shape", train_images.shape)
# 批量化和打乱数据
BUFFER_SIZE = 60000
BATCH_SIZE = 256
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

train_images shape (60000, 28, 28)


In [31]:
#训练
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# train_dataset.repeat(10) # 表示该训练集复制10次
for x_batch_train, y_batch_train in train_dataset:
  #取一个batch
  with tf.GradientTape() as tape:   
    logits = layer(x_batch_train)  
    loss_value = loss_fn(y_batch_train, logits)
  #？为什么在with外面
  grads = tape.gradient(loss_value, model.trainable_weights)
  optimizer.apply_gradients(zip(grads, model.trainable_weights))

ValueError: too many values to unpack (expected 2)